In [1]:
import sys; sys.path.append(r"C:\Users\Pierre\Documents\@ADMIN\#KUL - IA\_Stage\TexTGaiN\gab\Personnal Gab"); sys.path.append(r"C:\Users\Pierre\Documents\@ADMIN\#KUL - IA\@Classes\Methods of Corpus Linguistics\FINAL PAPER")
import os
from grasp import grasp as g
import pattern.metrics as p 
import pickle
from collections import Counter, defaultdict
from itertools import chain
from pprint import pprint
import csv

In [2]:
REDDIT_POSTS = "reddit_posts"
GAB_POSTS = "gab_posts"

In [3]:
def kw_cont_table(kw, tc, rc):
	# if type(rc) or type(tc) != collections.Counter:

	# 	raise TypeError("Must pass a Counter object for the corpora.")
	"""

	outputs a contingency table for the keyword "kw", in the target corpus "tc" and the reference corpus "rc"
	"""
	row1 = [tc.get(kw, 0), sum(tc.values()) - tc.get(kw, 0)]
	row2 = [rc.get(kw, 0), sum(rc.values()) - rc.get(kw, 0)]
 
	return [row1[0], row1[1], row2[0], row2[1]]





In [4]:
def exp_freq_table(cont):
	"""
	outputs a frequency table for the keyword "kw", for the target corpus "tc" and the reference corpus "rc" (cf. kw_cont_table)
	"""	
	r1 = cont[0] + cont[1]
	r2 = cont[2] + cont[3]
	c1 = cont[0] + cont[2]
	c2 = cont[1] + cont[3]

	n = r1 + r2
	
	a = (r1 * c1)/n
	b = (r1 * c2)/n
	c = (r2 * c1)/n
	d = (r2 * c2)/n

	return [a, b, c, d]

In [5]:
def kw_pmi(kw, tc, rc, freq_filter = 5):
	#computes pmi value for keywords
	
	cont_table = kw_cont_table(kw, tc, rc)
	freq_table = exp_freq_table(cont_table)

	if cont_table[0] >= freq_filter and kw in rc and kw in tc:
		return math.log2(cont_table[0]/freq_table[0])
	else:
		return 0.0

In [6]:
def save(data, file="temp", path = os.getcwd()):
	"""
		Pickles a file more easily. 
	"""
	with open(os.path.join(path, file), "wb") as f:
		pickle.dump(data, f)

In [7]:
def load(file, path = os.getcwd()):
	"""
		Loads a pickled file more easily. 
	"""
	whole_path = os.path.join(path, file)
	with open(whole_path, "rb") as f:
		data = pickle.load(f)
	return data

Loading datasets

In [8]:
gab = []
for file in os.listdir(GAB_POSTS):
        gab.extend(g.csv(g.cd(GAB_POSTS, file)))
reddit  = []
for file in os.listdir(REDDIT_POSTS):
        reddit.extend(g.csv(g.cd(REDDIT_POSTS, file)))

In [9]:
gab, reddit = [e[0] for e in gab], [e[0] for e in reddit]

Pre-Processing

In [6]:
ref = reddit 
targ = gab
ref, targ = list(chain(ref)), list(chain(targ)) #joins all lists in one list
ref, targ = " ".join(ref), " ".join(targ) #joins posts in one string
ref, targ = g.tokenize(ref, language="en"), g.tokenize(targ, language="en")
ref, targ = ref.lower(), targ.lower()
ref, targ = ref.split(), targ.split() #split by word
ref, targ = Counter(ref), Counter(targ)
# print(g0.most_common(3))

<font size=26>#################################</font>

In [ ]:
sorted_targpmi = defaultdict()
sorted_targpmi = {word:kw_pmi(word,targ, ref, freq_filter = 5) for word in targ}

In [22]:
sorted_targpmi = [(k, pmi[0]) for k,pmi in targpmi.items()]#sorted by value
sorted_targpmi = sorted(sorted_targpmi, key=lambda x:x[1], reverse=True)

In [27]:
pow_format = g.csv("pmi_reddit_gab_posts.csv")
pow_format.append(["WORD", "PMI"])
pow_format.update(sorted_targpmi)
pow_format.save()